# Backtest y Extensiones del Modelo

**Notebook 3 de 3** | Aplicación Práctica y Simulación

En este notebook:
1. Definimos una estrategia simple basada en regímenes
2. Implementamos backtest con walk-forward validation
3. Comparamos contra Buy & Hold
4. Generamos series sintéticas con la matriz de transición
5. Discutimos extensiones futuras

---

## 0. Setup y Carga de Datos

In [ ]:
# Para Google Colab: descomentar y ejecutar
# !pip install -q yfinance statsmodels networkx
# !git clone https://github.com/tu-usuario/blog_regimenes.git
# %cd blog_regimenes/notebooks

In [ ]:
import sys
sys.path.append("..")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta

from config import *
from src.data import download_spy_data
from src.features import compute_oliva_features
from src.regimes import standardize_features, fit_kmeans, compute_transition_matrix
from src.plotting import setup_plotting_style, save_figure, plot_equity_curves

setup_plotting_style()

SAVE_FIGURES = True
FIGURES_PATH = "../figures/post3/"

print(f"Walk-forward: re-entrenar cada {WALK_FORWARD_RETRAIN_MONTHS} meses")
print(f"Ventana de entrenamiento: {WALK_FORWARD_WINDOW_YEARS} años")

In [ ]:
# TODO: Descargar datos completos
# df = download_spy_data()

---
## 1. Definición de la Estrategia

### Reglas simples:
- **Régimen favorable (Bull Tranquilo, Bull Volátil):** 100% invertido en SPY
- **Régimen desfavorable (Bear Volátil, Neutro):** 100% en cash (0% exposición)

Esta es una estrategia simplificada para demostración. En producción se considerarían:
- Costos de transacción
- Slippage
- Posiciones parciales
- Reglas de confirmación

In [ ]:
# Definir qué regímenes son "favorables" para estar invertido
# Esto se ajustará después de ver los centroides
FAVORABLE_REGIMES = [0, 3]  # Bull Tranquilo, Bull Volátil
UNFAVORABLE_REGIMES = [1, 2]  # Bear Volátil, Neutro

def get_position(regime: int) -> float:
    """
    Determina la posición basada en el régimen.
    
    Returns
    -------
    float : 1.0 si invertido, 0.0 si en cash
    """
    return 1.0 if regime in FAVORABLE_REGIMES else 0.0

---
## 2. Walk-Forward Validation

Para evitar look-ahead bias, implementamos walk-forward:
1. Entrenar modelo con datos de los últimos N años
2. Aplicar al siguiente periodo (M meses)
3. Re-entrenar y repetir

Parámetros:
- **Ventana de entrenamiento:** 5 años
- **Re-entrenamiento:** cada 12 meses

In [ ]:
def walk_forward_backtest(
    df: pd.DataFrame,
    train_window_years: int = WALK_FORWARD_WINDOW_YEARS,
    retrain_months: int = WALK_FORWARD_RETRAIN_MONTHS,
    start_date: str = "2005-01-01",  # Empezar después de tener suficiente historia
) -> pd.DataFrame:
    """
    Ejecuta backtest con walk-forward validation.
    
    Parameters
    ----------
    df : pd.DataFrame
        DataFrame con datos y columna 'returns'
    train_window_years : int
        Años de datos para entrenar
    retrain_months : int
        Frecuencia de re-entrenamiento en meses
    start_date : str
        Fecha de inicio del backtest
        
    Returns
    -------
    pd.DataFrame
        DataFrame con columnas: returns, regime, position, strategy_returns
    """
    results = []
    
    # Fechas de re-entrenamiento
    start = pd.Timestamp(start_date)
    end = df.index[-1]
    
    current_date = start
    model = None
    scaler = None
    
    while current_date < end:
        # Definir ventana de entrenamiento
        train_start = current_date - relativedelta(years=train_window_years)
        train_end = current_date
        
        # Definir ventana de test (hasta próximo re-entrenamiento)
        test_end = min(current_date + relativedelta(months=retrain_months), end)
        
        # Extraer datos
        df_train = df.loc[train_start:train_end]
        df_test = df.loc[train_end:test_end].iloc[1:]  # Excluir fecha de corte
        
        if len(df_train) < 252 or len(df_test) == 0:  # Mínimo 1 año de datos
            current_date = test_end
            continue
        
        # Calcular features y entrenar
        features_train = compute_oliva_features(df_train)
        features_scaled, scaler = standardize_features(features_train)
        model, _ = fit_kmeans(features_scaled, k=K_REGIMES)
        
        # Aplicar a test
        features_test = compute_oliva_features(df_test)
        
        # Alinear índices (features tienen menos filas por el rolling)
        common_idx = features_test.index.intersection(df_test.index)
        if len(common_idx) == 0:
            current_date = test_end
            continue
            
        features_test_scaled, _ = standardize_features(features_test.loc[common_idx], scaler=scaler)
        labels_test = model.predict(features_test_scaled)
        
        # Calcular posiciones y retornos
        for i, (idx, row) in enumerate(df_test.loc[common_idx].iterrows()):
            regime = labels_test[i]
            position = get_position(regime)
            strategy_return = position * row['returns']
            
            results.append({
                'date': idx,
                'returns': row['returns'],
                'regime': regime,
                'position': position,
                'strategy_returns': strategy_return,
                'train_end': train_end,
            })
        
        # Avanzar al siguiente periodo
        current_date = test_end
        print(f"Periodo {train_end.strftime('%Y-%m')} -> {test_end.strftime('%Y-%m')}: {len(df_test.loc[common_idx])} días")
    
    return pd.DataFrame(results).set_index('date')

# TODO: Ejecutar backtest
# backtest_results = walk_forward_backtest(df)

---
## 3. Cálculo de Equity Curves

In [ ]:
def compute_equity_curves(backtest_df: pd.DataFrame, initial_capital: float = 10000) -> tuple:
    """
    Calcula curvas de equity para estrategia y buy & hold.
    
    Returns
    -------
    tuple : (equity_strategy, equity_benchmark)
    """
    # Estrategia
    cumulative_strategy = (1 + backtest_df['strategy_returns']).cumprod()
    equity_strategy = initial_capital * cumulative_strategy
    
    # Buy & Hold
    cumulative_bh = (1 + backtest_df['returns']).cumprod()
    equity_benchmark = initial_capital * cumulative_bh
    
    return equity_strategy, equity_benchmark

# TODO: Calcular equity curves
# equity_strategy, equity_benchmark = compute_equity_curves(backtest_results)
# 
# save_path = f"{FIGURES_PATH}equity_curves.png" if SAVE_FIGURES else None
# plot_equity_curves(equity_strategy, equity_benchmark, save_path=save_path)

---
## 4. Métricas de Performance

In [ ]:
def compute_metrics(returns: pd.Series, risk_free_rate: float = 0.02) -> dict:
    """
    Calcula métricas de performance.
    
    Parameters
    ----------
    returns : pd.Series
        Serie de retornos diarios
    risk_free_rate : float
        Tasa libre de riesgo anual
        
    Returns
    -------
    dict : Diccionario con métricas
    """
    # Retorno total
    total_return = (1 + returns).prod() - 1
    
    # CAGR
    n_years = len(returns) / 252
    cagr = (1 + total_return) ** (1 / n_years) - 1 if n_years > 0 else 0
    
    # Volatilidad anualizada
    volatility = returns.std() * np.sqrt(252)
    
    # Sharpe Ratio
    excess_return = cagr - risk_free_rate
    sharpe = excess_return / volatility if volatility > 0 else 0
    
    # Max Drawdown
    cumulative = (1 + returns).cumprod()
    rolling_max = cumulative.expanding().max()
    drawdown = (cumulative - rolling_max) / rolling_max
    max_drawdown = drawdown.min()
    
    # Calmar Ratio
    calmar = cagr / abs(max_drawdown) if max_drawdown != 0 else 0
    
    return {
        'Total Return': f"{total_return:.2%}",
        'CAGR': f"{cagr:.2%}",
        'Volatility (Ann.)': f"{volatility:.2%}",
        'Sharpe Ratio': f"{sharpe:.2f}",
        'Max Drawdown': f"{max_drawdown:.2%}",
        'Calmar Ratio': f"{calmar:.2f}",
    }

# TODO: Comparar métricas
# metrics_strategy = compute_metrics(backtest_results['strategy_returns'])
# metrics_benchmark = compute_metrics(backtest_results['returns'])
# 
# comparison = pd.DataFrame({
#     'Estrategia': metrics_strategy,
#     'Buy & Hold': metrics_benchmark
# })
# comparison

In [ ]:
# TODO: Métricas adicionales
# pct_time_invested = backtest_results['position'].mean()
# n_trades = (backtest_results['position'].diff().abs() > 0).sum()
# 
# print(f"% Tiempo invertido: {pct_time_invested:.1%}")
# print(f"Número de cambios de posición: {n_trades}")

---
## 5. Análisis de Drawdown

In [ ]:
def plot_drawdown_comparison(backtest_df: pd.DataFrame, save_path=None):
    """
    Compara drawdown de estrategia vs benchmark.
    """
    fig, ax = plt.subplots(figsize=(14, 6))
    
    for label, returns_col, color in [
        ('Estrategia', 'strategy_returns', '#2ecc71'),
        ('Buy & Hold', 'returns', '#3498db')
    ]:
        cumulative = (1 + backtest_df[returns_col]).cumprod()
        rolling_max = cumulative.expanding().max()
        drawdown = (cumulative - rolling_max) / rolling_max
        
        ax.fill_between(drawdown.index, drawdown.values, 0, 
                       alpha=0.4, label=label, color=color)
    
    ax.set_xlabel('Fecha')
    ax.set_ylabel('Drawdown')
    ax.set_title('Drawdown: Estrategia vs Buy & Hold')
    ax.legend()
    ax.yaxis.set_major_formatter(plt.FuncFormatter(lambda y, _: f'{y:.0%}'))
    
    plt.tight_layout()
    
    if save_path:
        save_figure(fig, save_path)
    
    return fig

# TODO: Ejecutar
# save_path = f"{FIGURES_PATH}drawdown_comparacion.png" if SAVE_FIGURES else None
# plot_drawdown_comparison(backtest_results, save_path)

---
## 6. Limitación: Evaluación Retrospectiva

Incluso con walk-forward, hay limitaciones:

1. **Selection bias:** Elegimos SPY y el periodo de análisis después de ver los datos
2. **Parámetros:** K=4, ventanas, definición de regímenes favorables
3. **Costos ignorados:** Transacciones, slippage, impuestos

Para uso real, se recomienda:
- Paper trading antes de capital real
- Análisis de sensibilidad a parámetros
- Simulación de costos realistas

---
## 7. Generación de Series Sintéticas

Usamos la matriz de transición para simular secuencias de regímenes y generar retornos sintéticos.

In [ ]:
def simulate_regime_sequence(
    trans_matrix: np.ndarray,
    n_steps: int,
    initial_regime: int = 0,
    random_state: int = None,
) -> np.ndarray:
    """
    Simula secuencia de regímenes usando la matriz de transición.
    
    Parameters
    ----------
    trans_matrix : np.ndarray
        Matriz de transición (k x k)
    n_steps : int
        Número de pasos a simular
    initial_regime : int
        Régimen inicial
    random_state : int
        Semilla para reproducibilidad
        
    Returns
    -------
    np.ndarray : Secuencia de regímenes
    """
    if random_state is not None:
        np.random.seed(random_state)
    
    k = len(trans_matrix)
    regimes = [initial_regime]
    
    for _ in range(n_steps - 1):
        current = regimes[-1]
        probs = trans_matrix[current]
        next_regime = np.random.choice(k, p=probs)
        regimes.append(next_regime)
    
    return np.array(regimes)


def generate_synthetic_returns(
    regime_sequence: np.ndarray,
    regime_params: dict,
    random_state: int = None,
) -> np.ndarray:
    """
    Genera retornos sintéticos basados en la secuencia de regímenes.
    
    Parameters
    ----------
    regime_sequence : np.ndarray
        Secuencia de regímenes
    regime_params : dict
        {regime: {'mean': μ, 'std': σ}} parámetros por régimen
    random_state : int
        Semilla para reproducibilidad
        
    Returns
    -------
    np.ndarray : Retornos sintéticos
    """
    if random_state is not None:
        np.random.seed(random_state)
    
    returns = []
    for regime in regime_sequence:
        params = regime_params[regime]
        r = np.random.normal(params['mean'], params['std'])
        returns.append(r)
    
    return np.array(returns)

# TODO: Estimar parámetros por régimen de los datos reales
# regime_params = {}
# for regime in range(K_REGIMES):
#     regime_returns = df_train.loc[labels == regime, 'returns']
#     regime_params[regime] = {
#         'mean': regime_returns.mean(),
#         'std': regime_returns.std()
#     }

In [ ]:
# TODO: Generar múltiples series sintéticas
# n_simulations = 100
# n_steps = len(df_train)
# 
# synthetic_series = []
# for i in range(n_simulations):
#     regime_seq = simulate_regime_sequence(trans_matrix, n_steps, random_state=i)
#     returns_sim = generate_synthetic_returns(regime_seq, regime_params, random_state=i+1000)
#     synthetic_series.append(returns_sim)

---
## 8. Comparación: Series Reales vs Sintéticas

In [ ]:
def compare_real_vs_synthetic(real_returns, synthetic_series, save_path=None):
    """
    Compara propiedades estadísticas de retornos reales vs sintéticos.
    """
    from statsmodels.graphics.tsaplots import acf
    
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    
    # 1. Histograma de retornos
    ax = axes[0, 0]
    ax.hist(real_returns, bins=50, density=True, alpha=0.7, label='Real', color='blue')
    for i, syn in enumerate(synthetic_series[:5]):
        ax.hist(syn, bins=50, density=True, alpha=0.2, color='red')
    ax.hist(synthetic_series[0], bins=50, density=True, alpha=0.5, label='Sintético', color='red')
    ax.set_title('Distribución de Retornos')
    ax.legend()
    ax.set_xlim(-0.1, 0.1)
    
    # 2. ACF de retornos
    ax = axes[0, 1]
    acf_real = acf(real_returns.dropna(), nlags=30)
    ax.bar(range(len(acf_real)), acf_real, alpha=0.7, label='Real', color='blue', width=0.4)
    acf_syn = np.mean([acf(s, nlags=30) for s in synthetic_series], axis=0)
    ax.bar(np.arange(len(acf_syn)) + 0.4, acf_syn, alpha=0.7, label='Sintético (media)', color='red', width=0.4)
    ax.set_title('ACF de Retornos')
    ax.legend()
    ax.set_xlabel('Lag')
    
    # 3. ACF de |retornos|
    ax = axes[1, 0]
    acf_real_abs = acf(np.abs(real_returns.dropna()), nlags=30)
    ax.bar(range(len(acf_real_abs)), acf_real_abs, alpha=0.7, label='Real', color='blue', width=0.4)
    acf_syn_abs = np.mean([acf(np.abs(s), nlags=30) for s in synthetic_series], axis=0)
    ax.bar(np.arange(len(acf_syn_abs)) + 0.4, acf_syn_abs, alpha=0.7, label='Sintético (media)', color='red', width=0.4)
    ax.set_title('ACF de |Retornos| (Volatility Clustering)')
    ax.legend()
    ax.set_xlabel('Lag')
    
    # 4. Tabla de momentos
    ax = axes[1, 1]
    ax.axis('off')
    
    moments_real = [
        real_returns.mean(), real_returns.std(), 
        real_returns.skew(), real_returns.kurtosis()
    ]
    moments_syn = [
        np.mean([s.mean() for s in synthetic_series]),
        np.mean([s.std() for s in synthetic_series]),
        np.mean([pd.Series(s).skew() for s in synthetic_series]),
        np.mean([pd.Series(s).kurtosis() for s in synthetic_series]),
    ]
    
    table_data = [
        ['', 'Real', 'Sintético'],
        ['Media', f'{moments_real[0]:.6f}', f'{moments_syn[0]:.6f}'],
        ['Std', f'{moments_real[1]:.6f}', f'{moments_syn[1]:.6f}'],
        ['Skewness', f'{moments_real[2]:.4f}', f'{moments_syn[2]:.4f}'],
        ['Kurtosis', f'{moments_real[3]:.4f}', f'{moments_syn[3]:.4f}'],
    ]
    
    table = ax.table(cellText=table_data, loc='center', cellLoc='center')
    table.auto_set_font_size(False)
    table.set_fontsize(12)
    table.scale(1.2, 1.8)
    ax.set_title('Comparación de Momentos')
    
    plt.tight_layout()
    
    if save_path:
        save_figure(fig, save_path)
    
    return fig

# TODO: Ejecutar comparación
# save_path = f"{FIGURES_PATH}real_vs_sintetico.png" if SAVE_FIGURES else None
# compare_real_vs_synthetic(df_train['returns'], synthetic_series, save_path)

### Interpretación

TODO: Discutir qué propiedades captura el modelo y cuáles no:
- ¿Las series sintéticas tienen volatility clustering?
- ¿Capturan el exceso de kurtosis?
- Limitaciones del enfoque de regímenes i.i.d. dentro de cada régimen

---
## 9. Conclusiones y Extensiones Futuras

### Resumen de resultados

TODO: Completar con resultados reales:
- Performance de la estrategia vs buy & hold
- ¿El modelo añade valor después de costos?
- Robustez de los resultados

### Extensiones posibles

1. **Jump Models / Hidden Semi-Markov Models**
   - Modelan explícitamente la duración de los regímenes
   - Resuelven la limitación de "sin memoria de duración"

2. **Predicción de régimen con ML**
   - Usar features adicionales (VIX, spreads de crédito, etc.)
   - Random Forest / XGBoost para predecir el régimen del siguiente día

3. **Ensemble de modelos**
   - Combinar K-Means con otros indicadores
   - Reducir dependencia de un solo modelo

4. **Optimización de la estrategia**
   - Posiciones parciales según confianza del régimen
   - Reglas de confirmación (esperar N días en nuevo régimen)

**Próximos pasos del TFM:** Implementar jump models y comparar con el enfoque actual.